In [1]:
import tensorflow as tf
from tensorflow.keras import losses, optimizers, mixed_precision

from drivable.data import BDDSemanticSegmentationLoader
from drivable.model import build_deeplabv3_plus

In [2]:
mixed_precision.set_global_policy("mixed_float16")
policy = mixed_precision.global_policy()

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0


In [3]:
data_loader = BDDSemanticSegmentationLoader(data_dir="../artifacts/bdd100k-dataset:v0")
train_dataset, val_dataset = data_loader.get_datasets()
print(train_dataset.element_spec, val_dataset.element_spec)

(TensorSpec(shape=(16, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(16, 512, 512, 1), dtype=tf.float32, name=None)) (TensorSpec(shape=(16, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(16, 512, 512, 1), dtype=tf.float32, name=None))


In [4]:
model = build_deeplabv3_plus(image_size=512, num_classes=20)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
296/437 [===================>..........] - ETA: 37s - loss: nan - accuracy: 0.0714